In [1]:
import mne
import numpy as np
import pandas as pd
import keras
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow import keras
from keras.models import Sequential, Model
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, SimpleRNN
from keras.utils import to_categorical
from keras import regularizers, layers, Input, Model
import keras_nlp
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
import torch
import evaluate

/opt/anaconda3/envs/eeg2text/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
epochs = mne.read_epochs("Murphy et al Dataset/session0_files0-151-epo/rsvp_session0_files135-142-epo.fif")
eeg_data = epochs.get_data() # NumPy array of shape [n_trials, n_electrodes, n_timepoints] with n_electrodes = 64, n_timepoints = 276 and n_trials is variable by each file.
metadata = epochs.metadata   # Pandas DataFrame

Reading /Users/raygroshan/EEG2Text/Murphy et al Dataset/session0_files0-151-epo/rsvp_session0_files135-142-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =    -200.00 ...     900.00 ms
        0 CTF compensation matrices available
Adding metadata with 16 columns
6119 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [5]:
epochs_df = epochs.to_data_frame(index=["epoch"])

In [6]:
epochs_df.head()

,time,condition,Fp1,Fz,F3,F7,FT9,FC5,FC1,C3,...,TP8,C6,C2,FC4,FT8,F6,AF8,AF4,F2,FCz
epoch,,,,,,,,,,,,,,,,,,,,,
4,-0.200,SCONJ,-8.164026,-7.718395,-1.637494,14.729168,21.816702,8.618022,-2.829288,4.991280,...,7.279476,2.708715,-1.774597,-10.398217,-0.720028,-14.961451,-23.712768,-15.322685,-9.937602,-6.278211
4,-0.196,SCONJ,-6.728718,-6.621864,-0.899448,17.182030,22.738490,8.131625,-3.978947,1.889640,...,9.956419,3.285817,-2.143940,-7.966863,2.064288,-12.381566,-24.517216,-13.111813,-7.460168,-5.812950
4,-0.192,SCONJ,-6.859263,-6.093499,-0.613570,17.967430,21.929811,7.183872,-5.645842,-1.435819,...,12.939852,4.275948,-3.368283,-4.386669,5.420504,-8.751219,-24.299155,-11.186612,-5.507703,-5.943257
4,-0.188,SCONJ,-8.504260,-6.247392,-1.030414,16.845115,20.332367,6.385915,-7.047753,-3.395246,...,15.244222,5.186755,-5.465576,-1.483275,7.687897,-6.227280,-24.166953,-10.504291,-4.837189,-6.627208
4,-0.184,SCONJ,-10.605402,-7.011631,-2.010493,14.530716,18.866647,5.878261,-7.629260,-3.639891,...,16.508895,5.741850,-7.750016,-0.550401,8.063190,-5.776487,-24.696328,-11.088340,-5.605135,-7.658965


In [9]:
metadata.head()

,trigger,word,pos,sess,sent_num,filename,len,freq,sess_id,prev_pos,next_pos,prev_freq,next_freq,prev_len,next_len,sent_ident
4,2,as,SCONJ,sess_0,sent_0,weblog-blogspot.com_dakbangla_20050210141134_E...,2,6.79,session0_files135-142_triggers_processed.txt,SYM,DET,0.00,7.77,1.0,3.0,sess_0_weblog-blogspot.com_dakbangla_200502101...
5,3,the,DET,sess_0,sent_0,weblog-blogspot.com_dakbangla_20050210141134_E...,3,7.77,session0_files135-142_triggers_processed.txt,SCONJ,NOUN,6.79,4.20,2.0,12.0,sess_0_weblog-blogspot.com_dakbangla_200502101...
6,4,implications,NOUN,sess_0,sent_0,weblog-blogspot.com_dakbangla_20050210141134_E...,12,4.20,session0_files135-142_triggers_processed.txt,DET,ADP,7.77,7.45,3.0,2.0,sess_0_weblog-blogspot.com_dakbangla_200502101...
7,5,of,ADP,sess_0,sent_0,weblog-blogspot.com_dakbangla_20050210141134_E...,2,7.45,session0_files135-142_triggers_processed.txt,NOUN,DET,4.20,7.77,12.0,3.0,sess_0_weblog-blogspot.com_dakbangla_200502101...
8,6,the,DET,sess_0,sent_0,weblog-blogspot.com_dakbangla_20050210141134_E...,3,7.77,session0_files135-142_triggers_processed.txt,ADP,PROPN,7.45,2.64,2.0,7.0,sess_0_weblog-blogspot.com_dakbangla_200502101...


In [12]:
combined_df = epochs_df.join(metadata)

In [13]:
combined_df.head()

,time,condition,Fp1,Fz,F3,F7,FT9,FC5,FC1,C3,...,len,freq,sess_id,prev_pos,next_pos,prev_freq,next_freq,prev_len,next_len,sent_ident
4,-0.200,SCONJ,-8.164026,-7.718395,-1.637494,14.729168,21.816702,8.618022,-2.829288,4.991280,...,2,6.79,session0_files135-142_triggers_processed.txt,SYM,DET,0.0,7.77,1.0,3.0,sess_0_weblog-blogspot.com_dakbangla_200502101...
4,-0.196,SCONJ,-6.728718,-6.621864,-0.899448,17.182030,22.738490,8.131625,-3.978947,1.889640,...,2,6.79,session0_files135-142_triggers_processed.txt,SYM,DET,0.0,7.77,1.0,3.0,sess_0_weblog-blogspot.com_dakbangla_200502101...
4,-0.192,SCONJ,-6.859263,-6.093499,-0.613570,17.967430,21.929811,7.183872,-5.645842,-1.435819,...,2,6.79,session0_files135-142_triggers_processed.txt,SYM,DET,0.0,7.77,1.0,3.0,sess_0_weblog-blogspot.com_dakbangla_200502101...
4,-0.188,SCONJ,-8.504260,-6.247392,-1.030414,16.845115,20.332367,6.385915,-7.047753,-3.395246,...,2,6.79,session0_files135-142_triggers_processed.txt,SYM,DET,0.0,7.77,1.0,3.0,sess_0_weblog-blogspot.com_dakbangla_200502101...
4,-0.184,SCONJ,-10.605402,-7.011631,-2.010493,14.530716,18.866647,5.878261,-7.629260,-3.639891,...,2,6.79,session0_files135-142_triggers_processed.txt,SYM,DET,0.0,7.77,1.0,3.0,sess_0_weblog-blogspot.com_dakbangla_200502101...


In [14]:
combined_df['pos'] = combined_df['pos'].astype('category')
combined_df['pos'] = combined_df['pos'].cat.codes

combined_df['prev_pos'] = combined_df['prev_pos'].astype('category')
combined_df['prev_pos'] = combined_df['prev_pos'].cat.codes

combined_df['next_pos'] = combined_df['next_pos'].astype('category')
combined_df['next_pos'] = combined_df['next_pos'].cat.codes

In [15]:
# Create train/test split
X = combined_df[['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 
        'CP5', 'CP1', 'Pz', 'P3', 'P7','O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 
        'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 
        'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 
        'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 
        'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz', 'len', 
        'freq', 'prev_pos', 'next_pos', 'prev_freq', 'next_freq', 'prev_len', 'next_len' ]].values
y = combined_df['pos'].values

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state= 42)

In [17]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [18]:
x_train = X_train.reshape(X_train.shape[0], -1, X_train.shape[1])
x_test = X_test.reshape(X_test.shape[0], -1, X_test.shape[1])
y_train = y_train.reshape(y_train.shape[0], -1, y_train.shape[1])
y_test = y_test.reshape(y_test.shape[0], -1, y_test.shape[1])

### Simple RNN Test

In [19]:
model_rnn= Sequential()
model_rnn.add(SimpleRNN(64, return_sequences=True, input_shape=(x_train.shape[1],x_train.shape[2])))
model_rnn.add(SimpleRNN(64, return_sequences=True))
model_rnn.add(SimpleRNN(64, return_sequences=True))
model_rnn.add(Dropout(0.2))
model_rnn.add(Dense(y_train.shape[2], activation='softmax'))
model_rnn.summary()

# Train the model
opt = Adam(learning_rate=0.001)
model_rnn.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
trained_model_base = model_rnn.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

/opt/anaconda3/envs/eeg2text/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 1, 64)          │         8,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 1, 64)          │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 1, 64)          │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1, 16)          │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,320 (102.81 KB)

 Trainable params: 26,320 (102.81 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 30s 1ms/step - accuracy: 0.4290 - loss: 1.6378 - val_accuracy: 0.6675 - val_loss: 0.9574
Epoch 2/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.6705 - loss: 0.9569 - val_accuracy: 0.7482 - val_loss: 0.7239
Epoch 3/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 29s 1ms/step - accuracy: 0.7254 - loss: 0.7970 - val_accuracy: 0.7703 - val_loss: 0.6747
Epoch 4/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.7563 - loss: 0.7086 - val_accuracy: 0.8074 - val_loss: 0.5548
Epoch 5/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 29s 1ms/step - accuracy: 0.7765 - loss: 0.6519 - val_accuracy: 0.8263 - val_loss: 0.5114
Epoch 6/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 30s 1ms/step - accuracy: 0.7912 - loss: 0.6117 - val_accuracy: 0.8240 - val_loss: 0.5345
Epoch 7/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.8002 - loss: 0.5866 - val_accuracy: 0.8335 - val_loss: 0.4973
Epoch 8/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 

In [20]:
from sklearn.metrics import classification_report
y_pred = model_rnn.predict(x_test)
y_pred_m = y_pred.reshape(y_pred.shape[0], y_pred.shape[2])
y_test_m = y_test.reshape(y_test.shape[0], y_test.shape[2])
y_pred_m = np.argmax(y_pred_m, axis=1)
y_test_m = np.argmax(y_test_m, axis=1)
print(classification_report(y_test_m, y_pred_m))



13195/13195 ━━━━━━━━━━━━━━━━━━━━ 9s 705us/step
              precision    recall  f1-score   support

           0       0.82      0.75      0.78     29882
           1       0.89      0.88      0.88     44642
           2       0.78      0.72      0.75     23138
           3       0.82      0.87      0.84     28049
           4       0.90      0.86      0.88     17157
           5       0.94      0.94      0.94     41695
           6       0.78      0.88      0.83       353
           7       0.85      0.91      0.88     79898
           8       0.94      0.69      0.80      5124
           9       0.91      0.93      0.92      9954
          10       0.89      0.86      0.88     41198
          11       0.91      0.89      0.90     34683
          12       1.00      1.00      1.00      2858
          13       0.80      0.74      0.77      9617
          14       0.83      0.88      0.85     53432
          15       0.97      0.79      0.87       531

    accuracy                     

### LSTM Test

In [21]:
model_lstm = Sequential()
model_lstm.add(layers.LSTM(64, return_sequences=True, input_shape=(x_train.shape[1],x_train.shape[2])))
model_lstm.add(layers.LSTM(64, return_sequences=True))
model_lstm.add(layers.LSTM(64, return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(y_train.shape[2], activation='softmax'))
model_lstm.summary()

# Train the model
opt = Adam(learning_rate=0.001)
model_lstm.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
trained_model_base = model_lstm.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

/opt/anaconda3/envs/eeg2text/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 64)          │        35,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1, 64)          │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 1, 64)          │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1, 16)          │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,160 (399.06 KB)

 Trainable params: 102,160 (399.06 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 58s 3ms/step - accuracy: 0.4480 - loss: 1.5700 - val_accuracy: 0.6988 - val_loss: 0.8556
Epoch 2/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 57s 3ms/step - accuracy: 0.7120 - loss: 0.8170 - val_accuracy: 0.8178 - val_loss: 0.5186
Epoch 3/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 75s 4ms/step - accuracy: 0.7932 - loss: 0.5762 - val_accuracy: 0.8707 - val_loss: 0.3778
Epoch 4/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 63s 3ms/step - accuracy: 0.8414 - loss: 0.4430 - val_accuracy: 0.9037 - val_loss: 0.2838
Epoch 5/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 87s 4ms/step - accuracy: 0.8704 - loss: 0.3639 - val_accuracy: 0.9227 - val_loss: 0.2311
Epoch 6/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 86s 4ms/step - accuracy: 0.8879 - loss: 0.3183 - val_accuracy: 0.9337 - val_loss: 0.1978
Epoch 7/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 89s 4ms/step - accuracy: 0.9006 - loss: 0.2847 - val_accuracy: 0.9321 - val_loss: 0.2058
Epoch 8/10
19792/19792 ━━━━━━━━━━━━━━━━━━━━ 87s 4ms/step - accuracy: 

In [22]:
from sklearn.metrics import classification_report
y_pred = model_lstm.predict(x_test)
y_pred_m = y_pred.reshape(y_pred.shape[0], y_pred.shape[2])
y_test_m = y_test.reshape(y_test.shape[0], y_test.shape[2])
y_pred_m = np.argmax(y_pred_m, axis=1)
y_test_m = np.argmax(y_test_m, axis=1)
print(classification_report(y_test_m, y_pred_m))


13195/13195 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step
              precision    recall  f1-score   support

           0       0.95      0.93      0.94     29882
           1       0.96      0.97      0.96     44642
           2       0.95      0.93      0.94     23138
           3       0.95      0.95      0.95     28049
           4       0.98      0.96      0.97     17157
           5       0.98      0.99      0.98     41695
           6       0.98      0.87      0.92       353
           7       0.95      0.96      0.96     79898
           8       0.96      0.94      0.95      5124
           9       0.97      0.98      0.98      9954
          10       0.96      0.97      0.97     41198
          11       0.96      0.96      0.96     34683
          12       0.99      1.00      0.99      2858
          13       0.96      0.91      0.94      9617
          14       0.96      0.95      0.96     53432
          15       0.99      0.92      0.96       531

    accuracy                      